In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_dir = '/kaggle/input/skin-cancer-img-augmented/skincancerimg/train'
test_dir = '/kaggle/input/skin-cancer-img-augmented/skincancerimg/test'

# Image dimensions and batch size
IMG_SIZE = (380, 380)  # EfficientNetB4 input size
BATCH_SIZE = 32

In [3]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# Get number of classes
NUM_CLASSES = len(train_generator.class_indices)
NUM_CLASSES

Found 4500 images belonging to 9 classes.
Found 1080 images belonging to 9 classes.


9

In [4]:
# Load EfficientNetB4 model
base_model = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(380, 380, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)  # Helps prevent overfitting
output = Dense(NUM_CLASSES, activation="softmax")(x)

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# Define callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="min", restore_best_weights=True)

# Train model
EPOCHS = 25  # Adjust as needed
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


141/141 ━━━━━━━━━━━━━━━━━━━━ 495s 2s/step - accuracy: 0.3128 - loss: 1.8818 - val_accuracy: 0.3352 - val_loss: 1.9374
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.7195 - loss: 0.8308 - val_accuracy: 0.5417 - val_loss: 1.4394
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.8697 - loss: 0.4327 - val_accuracy: 0.6250 - val_loss: 1.2066
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.9278 - loss: 0.2307 - val_accuracy: 0.6481 - val_loss: 1.1291
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.9532 - loss: 0.1499 - val_accuracy: 0.6574 - val_loss: 1.1573
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.9482 - loss: 0.1487 - val_accuracy: 0.6463 - val_loss: 1.3339
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.9692 - loss: 0.0965 - val_accuracy: 0.6648 - val_loss: 1.2470
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.9642 - loss: 0.1050 - val_accuracy: 0.642